In [ ]:
!pip install chess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import chess

# Define the modified moves for pawns
def my_pawn_moves(board, from_square):
    # Pawns move diagonally one step
    moves = []
    piece = board.piece_at(from_square)
    if piece.color == chess.WHITE:
        target_squares = [from_square + 7, from_square + 9]
    else:
        target_squares = [from_square - 7, from_square - 9]

    for target_square in target_squares:
        if chess.square_rank(target_square) != chess.square_rank(from_square):
            continue
        if not board.is_empty(target_square):
            continue
        moves.append(chess.Move(from_square, target_square))

    return moves

def my_pawn_captures(board, from_square, to_square):
    # Pawns capture forward one step
    moves = []
    piece = board.piece_at(from_square)
    if piece.color == chess.WHITE:
        target_square = to_square + 8
    else:
        target_square = to_square - 8

    if chess.square_file(target_square) == chess.square_file(to_square):
        if board.is_empty(target_square):
            moves.append(chess.Move(from_square, target_square))
        elif board.piece_at(target_square).color != piece.color:
            moves.append(chess.Move(from_square, target_square))

    return moves

# Function to remove original pawn moves and captures
def remove_original_pawn_moves(legal_moves, board):
    for move in list(legal_moves):
        if board.piece_at(move.from_square).piece_type == chess.PAWN:
            if move.to_square == board.ep_square:
                legal_moves.remove(move)
                legal_moves.extend(my_pawn_captures(board, move.from_square, move.to_square))
            elif chess.square_file(move.from_square) != chess.square_file(move.to_square) and board.is_empty(move.to_square):
                legal_moves.remove(move)
                legal_moves.extend(my_pawn_moves(board, move.from_square))
            elif abs(move.to_square - move.from_square) == 16:
                legal_moves.remove(move)

    return legal_moves

# Implement the win condition
def check_win_condition(board):
    if board.is_checkmate():
        if board.turn == chess.WHITE:
            print("Black wins")
        else:
            print("White wins")
        return True
    elif board.can_claim_draw():
        print("Draw")
        return True
    else:
        return False

# Function to get all possible legal moves
"""def get_legal_moves(board):
    legal_moves = list(board.legal_moves)

    # Remove original pawn moves and captures
    legal_moves = remove_original_pawn_moves(legal_moves, board)

    return legal_moves"""

# Function to get all possible legal moves
def get_legal_moves(board):
    legal_moves = list(board.legal_moves)

    # Check for modified pawn moves and captures
    for move in list(legal_moves):
        piece = board.piece_at(move.from_square)
        if piece.piece_type == chess.PAWN:
            # Remove the original moves in which pawns move forward or capture diagonally
            if move.to_square == board.ep_square or chess.square_file(move.from_square) != chess.square_file(move.to_square):
                legal_moves.remove(move)
            # Remove the move in which pawns move two steps forward
            if abs(chess.square_rank(move.to_square) - chess.square_rank(move.from_square)) == 2:
                legal_moves.remove(move)
            # Add the modified moves in which pawns move diagonally one step and capture forward one step
            else:
                legal_moves.remove(move)
                legal_moves.extend(my_pawn_moves(board, move.from_square))
                legal_moves.extend(my_pawn_captures(board, move.from_square, move.to_square))

    return legal_moves



In [ ]:
def evaluate_move(board, move):
    piece = board.piece_at(move.from_square)
    target = board.piece_at(move.to_square)
    reward = 0
    
    # Check if the move results in a check on the opposite king
    if board.gives_check(move):
        reward += 1000

    # Check if a piece is captured
    if target is not None:
        reward += 100 + 10 * piece.piece_type

    # Move closer to the opponent's king
    if piece.color == chess.WHITE:
        king_square = board.king(chess.BLACK)
        distance_before = chess.square_distance(move.from_square, king_square)
        distance_after = chess.square_distance(move.to_square, king_square)
    else:
        king_square = board.king(chess.WHITE)
        distance_before = chess.square_distance(move.from_square, king_square)
        distance_after = chess.square_distance(move.to_square, king_square)

    if distance_after < distance_before:
        reward += 10

    # Move towards the center of the board
    center_square = chess.E4
    center_distance_before = chess.square_distance(move.from_square, center_square)
    center_distance_after = chess.square_distance(move.to_square, center_square)

    if center_distance_after < center_distance_before:
        reward += 5

    # Prioritize moves that can move more
    piece_value = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }
    reward += piece_value[piece.piece_type]

    return reward


In [ ]:
def remove_castling_moves(legal_moves):
    """Removes castling moves from a list of legal moves."""
    return [move for move in legal_moves if not move.has_castling_rights()]


In [ ]:
import chess

# create the chess board object
board = chess.Board(fen="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
#board = chess.Board(fen="r1bqkbnr/ppp1pppp/2n5/3p4/8/2P5/PP1PPPPP/RNBQKBNR w KQkq - 0 1")

# generate legal moves
legal_moves = list(board.generate_legal_moves())
#legal_moves = get_legal_moves(legal_moves)

# remove castling moves
#legal_moves = remove_castling_moves(legal_moves)

# evaluate moves
for move in legal_moves:
    score = evaluate_move(board, move)
    print(f"Move {move} scored {score}")


Move g1h3 scored 13
Move g1f3 scored 18
Move b1c3 scored 18
Move b1a3 scored 13
Move h2h3 scored 11
Move g2g3 scored 11
Move f2f3 scored 16
Move e2e3 scored 16
Move d2d3 scored 16
Move c2c3 scored 11
Move b2b3 scored 11
Move a2a3 scored 11
Move h2h4 scored 11
Move g2g4 scored 11
Move f2f4 scored 16
Move e2e4 scored 16
Move d2d4 scored 16
Move c2c4 scored 11
Move b2b4 scored 11
Move a2a4 scored 11


In [ ]:
def play(board):
  legal_moves = list(board.generate_legal_moves())
  d = dict()
  for move in legal_moves:
    score = evaluate_move(board, move)
    d.update({move: score})
  s = 0;
  for item in d:
    if d[item] > s:
      out_move = item
      s = d[item]
  return out_move

board = chess.Board(fen="r1bqkbnr/ppp1pppp/2n5/3p4/8/2P5/PP1PPPPP/RNBQKBNR w KQkq - 0 1")

print(play(board))

d1c2
